<a href="https://colab.research.google.com/github/rogercost/voice-bird-id/blob/master/train_synthrec_l1_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# prompt: Import a pickle file from my Google Drive into a pandas dataframe

import pandas as pd
import pickle

from google.colab import drive
drive.mount('/content/gdrive')

file_path = '/content/gdrive/My Drive/synthetic_recordings.pkl'

try:
  with open(file_path, 'rb') as file:
    df = pickle.load(file)

  if isinstance(df, pd.DataFrame):
    print("Pickle file loaded successfully into a pandas DataFrame.")
    # Now you can work with the DataFrame 'df'
    print(df.head()) # Example: Display first 5 rows
  else:
    print("The loaded object is not a pandas DataFrame.")

except FileNotFoundError:
  print(f"Error: File not found at '{file_path}'. Please check the file path.")
except Exception as e:
  print(f"An error occurred: {e}")


Mounted at /content/gdrive
Pickle file loaded successfully into a pandas DataFrame.
                           name                              family  \
0  Black-bellied Whistling-Duck  Ducks, Geese, and Swans (Anatidae)   
1  Black-bellied Whistling-Duck  Ducks, Geese, and Swans (Anatidae)   
2  Black-bellied Whistling-Duck  Ducks, Geese, and Swans (Anatidae)   
3  Black-bellied Whistling-Duck  Ducks, Geese, and Swans (Anatidae)   
4  Black-bellied Whistling-Duck  Ducks, Geese, and Swans (Anatidae)   

                                           embedding  \
0  [-0.005670225713402033, 0.03125672787427902, 0...   
1  [-0.03728988766670227, 0.025586403906345367, -...   
2  [-0.02664249949157238, 0.04850178211927414, 0....   
3  [-0.03514464199542999, 0.026959339156746864, 0...   
4  [-0.05087177827954292, 0.012625874020159245, 0...   

                                       raw_recording  
0  Okay, so I've got a duck here, good size, mayb...  
1  Okay, little bit better view now. Stand

In [5]:
X = pd.DataFrame(df['embedding'].tolist())
y = df['family']

X.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.005670,0.031257,0.024194,0.019065,-0.010642,0.010769,0.008473,0.029615,-0.012113,-0.045429,...,0.032933,-0.025351,-0.015219,0.011457,-0.056603,0.025815,-0.007675,-0.034715,0.034644,0.017511
1,-0.037290,0.025586,-0.023516,-0.001923,-0.018404,-0.025943,-0.007776,-0.024614,-0.024128,-0.001324,...,0.017275,0.001317,0.001869,0.001307,-0.048640,0.015560,0.001194,-0.014198,0.019428,0.045738
2,-0.026642,0.048502,0.036331,-0.027529,0.001051,0.004019,0.000209,-0.018907,-0.002155,-0.034641,...,0.001582,0.028831,-0.024261,-0.012579,-0.015458,0.036560,0.041218,0.000531,0.006413,-0.012738
3,-0.035145,0.026959,0.020185,-0.017619,-0.004901,0.020210,0.014663,-0.025786,0.002994,-0.048650,...,0.020207,-0.038682,0.001430,0.022430,-0.054511,0.012445,-0.027285,-0.018549,0.025421,0.040538
4,-0.050872,0.012626,0.017226,-0.031940,0.009782,-0.003299,-0.013046,-0.035392,0.000995,-0.019688,...,-0.018691,-0.000055,-0.042163,-0.018476,-0.020169,0.011124,-0.000321,-0.018614,-0.018989,-0.001611


In [6]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Assume X and y are already defined as follows:
# X = pd.DataFrame(df['embedding'].tolist())  # Features (embedding dimensions)
# y = df['descriptor1']  # Target (class to predict)

# Step 1: Encode the target variable (if it's categorical)
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Convert class labels to integers

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Step 3: Train the XGBoost model
# Convert data into DMatrix format (optimized for XGBoost)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set XGBoost parameters
params = {
    'objective': 'multi:softmax',  # For multi-class classification
    'num_class': len(label_encoder.classes_),  # Number of unique classes
    'max_depth': 6,  # Maximum depth of a tree
    'eta': 0.1,  # Learning rate
    'subsample': 0.8,  # Subsample ratio of the training instances
    'colsample_bytree': 0.8,  # Subsample ratio of columns when constructing each tree
    'eval_metric': 'mlogloss',  # Evaluation metric for multi-class classification
    'seed': 42  # Random seed for reproducibility
}

# Train the model
num_rounds = 100  # Number of boosting rounds
model = xgb.train(params, dtrain, num_rounds)

# Step 4: Make predictions on the test set
y_pred = model.predict(dtest)

# Step 5: Evaluate the model
# Convert encoded predictions back to original class labels
y_pred_labels = label_encoder.inverse_transform(y_pred.astype(int))

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Accuracy: 0.7987
Classification Report:


ValueError: Number of classes, 67, does not match size of target_names, 68. Try specifying the labels parameter

In [14]:
print(y_test)
print(len(y_test))
print(y_pred)
print(len(y_pred))
print(label_encoder.classes_[:5])
print(len(label_encoder.classes_))

import numpy as np
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_, labels=np.arange(0, len(label_encoder.classes_), 1)))

[17 64 48 ... 65 59 46]
1932
[ 4. 64. 48. ...  4. 59. 46.]
1932
['Albatrosses (Diomedeidae)' 'Auks, Murres, and Puffins (Alcidae)'
 'Barn Owls (Tytonidae)' 'Bitterns, Herons, and Allies (Ardeidae)'
 'Blackbirds (Icteridae)']
68
                                                                precision    recall  f1-score   support

                                     Albatrosses (Diomedeidae)       0.00      0.00      0.00         2
                           Auks, Murres, and Puffins (Alcidae)       0.70      0.78      0.74        45
                                         Barn Owls (Tytonidae)       1.00      0.80      0.89         5
                       Bitterns, Herons, and Allies (Ardeidae)       0.88      0.72      0.79        50
                                        Blackbirds (Icteridae)       0.65      0.68      0.66        75
                                 Boobies and Gannets (Sulidae)       0.00      0.00      0.00         5
                                       Bush

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me